## Data preparation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

raw_data = np.loadtxt(
    './data/mushrooms.csv',
    skiprows=1,
    delimiter=',',
    dtype='U1'
)

In [2]:
# Each column has a different sets of categorical values
# We create a dictionary for each column 
encoding = list(map(lambda col: dict(), range(raw_data.shape[1])))

In [3]:
def convert(raw_data):
    # This variable keeps track of the used code
    used_code = -1
    x_train = []
    y_train = []
    for i in range(len(raw_data)):
        item = raw_data[i]
        encoded = []

        # Encode the cell value with an integer code
        for j in range(len(item)):
            cell = item[j]

            # The first cell is the training result
            # 1 if 'e' (edible), 0 otherwise
            if j == 0:
                y_train.append(1 if cell == 'e' else 0)
                continue

            # Check if dictionary has definition for the cell value
            # if not, initialize it to the latest unused code
            if not cell in encoding[j]:
                used_code += 1
                encoding[j][cell] = used_code

            # Use the encoded value
            encoded_value = encoding[j][cell]
            encoded.append(encoded_value)
            
        x_train.append(encoded)
    return np.array(x_train), np.array(y_train)
    

In [4]:
x_train, y_train = convert(raw_data)

## Scikit-learn Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=10000)
logreg.fit(x_train, y_train)

LogisticRegression(max_iter=10000)

In [6]:
y_pred = logreg.predict(x_train)

In [7]:
y_pred[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 1])

In [13]:
def evaluate(y_pred, y_train):
    m = y_pred.shape[0]
    
    tp = 0
    fp = 0
    fn = 0
    
    for i in range(m):
        target = y_train[i]
        pred = y_pred[i]

        if (target == 1 and pred == 1) or (target == 0 and pred == 0):
            tp += 1
        elif target == 1 and pred == 0:
            fn += 1
        elif target == 0 and pred == 1:
            fp += 1
            
    tp_rate = round(tp / (len(y_train)) * 100, 2)
    fp_rate = round(fp / (len(y_train)) * 100, 2)
    fn_rate = round(fn / (len(y_train)) * 100, 2)

    print(f'Accuracy: {tp_rate}%')
    print(f'False positives: {fp_rate}%')
    print(f'False negatives: {fn_rate}%')
    
    return tp_rate, fp_rate, fn_rate
        

In [14]:
evaluate(y_pred, y_train)

Accuracy: 100.0%
False positives: 0.0%
False negatives: 0.0%


(100.0, 0.0, 0.0)

## TensorFlow Implementation

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
# Binary classification task needs only one single-unit layer
model = Sequential([
    Dense(1, activation='sigmoid')
])

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
)

In [12]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
254/254 [==============================] - 0s 674us/step - loss: 14.6300
Epoch 2/20
254/254 [==============================] - 0s 658us/step - loss: 2.9378
Epoch 3/20
254/254 [==============================] - 0s 645us/step - loss: 1.1434
Epoch 4/20
254/254 [==============================] - 0s 665us/step - loss: 0.6137
Epoch 5/20
254/254 [==============================] - 0s 637us/step - loss: 0.3414
Epoch 6/20
254/254 [==============================] - 0s 650us/step - loss: 0.1930
Epoch 7/20
254/254 [==============================] - 0s 691us/step - loss: 0.1290
Epoch 8/20
254/254 [==============================] - 0s 674us/step - loss: 0.0950
Epoch 9/20
254/254 [==============================] - 0s 633us/step - loss: 0.0731
Epoch 10/20
254/254 [==============================] - 0s 633us/step - loss: 0.0596
Epoch 11/20
254/254 [==============================] - 0s 621us/step - loss: 0.0510
Epoch 12/20
254/254 [==============================] - 0s 645us/step - loss: 0.0451


In [13]:
model.predict(x_train[0].reshape(1, x_train[0].shape[0]))

1/1 [==============================] - 0s 60ms/step


array([[0.17370944]], dtype=float32)

In [14]:
def predict(x_train, model):
    # number of features
    n = x_train[0].shape[0]
    pred = []

    ten_pc = round(len(x_train)/10)
    
    for i, x in enumerate(x_train):
        y = model.predict(x.reshape(1, n), verbose=0)
        pred.append(1 if y >= 0.5 else 0)

        if i%ten_pc == 0:
            print(f'{round(i/ten_pc*10, 3)}% done')
    print('Finished')
    return np.array(pred)        

In [15]:
y_pred_tf = predict(x_train, model)

0% done
10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done
Finished


In [16]:
evaluate(y_pred_tf, y_train)

Accuracy: 100%
False positives: 0%
False negatives: 0%


(100, 0, 0)

## Numpy implementation using NN's weights

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 23        
                                                                 
Total params: 23 (92.00 Byte)
Trainable params: 23 (92.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
layer = model.get_layer('dense')
w, b = layer.get_weights()
print(w, b)

[[ 2.5702571e-04]
 [ 1.5553305e-02]
 [-4.9238619e-03]
 [ 4.2984885e-01]
 [-3.4207046e-01]
 [ 1.7485227e-01]
 [ 1.3823950e-01]
 [ 7.6096708e-01]
 [-4.1990492e-02]
 [ 1.1169981e-01]
 [ 1.1099790e-01]
 [-1.1303192e-01]
 [-3.3313517e-02]
 [-2.8390538e-02]
 [-6.7457347e-03]
 [-2.1528155e-01]
 [-2.7276477e-01]
 [-2.8869358e-03]
 [-4.9366575e-02]
 [-7.3915854e-02]
 [ 4.4026230e-02]
 [ 6.8315126e-02]] [-0.5693278]


In [30]:
w_t = w[:, 0]
b_t = b[0]
print(w_t, b_t)

[ 2.5702571e-04  1.5553305e-02 -4.9238619e-03  4.2984885e-01
 -3.4207046e-01  1.7485227e-01  1.3823950e-01  7.6096708e-01
 -4.1990492e-02  1.1169981e-01  1.1099790e-01 -1.1303192e-01
 -3.3313517e-02 -2.8390538e-02 -6.7457347e-03 -2.1528155e-01
 -2.7276477e-01 -2.8869358e-03 -4.9366575e-02 -7.3915854e-02
  4.4026230e-02  6.8315126e-02] -0.5693278


In [79]:
sigmoid = lambda z: 1/(1 + np.exp(-z))
y_pred_np = np.array(list(map(lambda x: 1 if sigmoid(w_t @ x + b) >= 0.5 else 0, x_train)))

In [80]:
evaluate(y_pred_np, y_train)

Accuracy: 100%
False positives: 0%
False negatives: 0%


(100, 0, 0)